In [1]:
# 엑스레이 이미지 파일 변환ver0.2 -> 이미지 짤리는 현상 보안

import shutil # 파일, 폴더 복사 모듈
import cv2 # openCV 모듈 
import os
import numpy as np
import pandas as pd

df = pd.read_excel('C:/Users/user/Desktop/[NIA] Runlist_1cycle_E3S690_Rev1_Draft.xlsx', header=None) # 엑셀 파일 위치 지정

# 디렉토리 설정
DIR_DESKTOP = "C:/Users/user/Desktop"
DIR_WORKING = DIR_DESKTOP + "/X-ray Data" # 촬영 데이터가 모여져 있는 폴더 경로
DIR_WORKING_COPY = DIR_WORKING + '/temp' # 변환 중간 이미지 저장 폴더
DIR_ROOT = DIR_DESKTOP + '/NIA'# 변환된 이미지 결과 저장 ROOT 폴더(NIA)

# 변환할 이미지 크기 설정
HEIGHT = 760
WIDTH = 896

i = 1 # 파일명 찾는 가변 변수 - 엑셀 시트의 시작 번호

if not os.path.exists(DIR_WORKING_COPY):
    shutil.copytree(DIR_WORKING, DIR_WORKING_COPY) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
if not os.path.exists(DIR_ROOT):
    os.mkdir(DIR_ROOT)

filelist = os.listdir(DIR_WORKING_COPY)

for file in filelist:

    # raw 이미지 -> png 이미지로 전체 크기로 바꾸기 코드
    os.chdir(DIR_WORKING_COPY) # change directory
    
    read_binary = open(DIR_WORKING_COPY+'/'+file, "rb") # 1. 파일 경로, 2. read + binary (rb)
    img = np.fromfile(read_binary, dtype='uint16', sep="")
    read_binary.close()

    img = np.reshape(img, [img.shape[0]//WIDTH, WIDTH]) # height -> 전체 count // width, 896 -> width
    
    markfile = df.values[i][10] + '.png' # image 파일 저장 이름 
    
    cv2.imwrite(markfile, img)

    # 파일 변환 확인용
    print(file)
    print(markfile)
    print(df.values[i][2])

    # 896, 760 크기로 변환 + 물품 분류 체계에 따라 폴더 나눠서 저장
    # 폴더 구조 : NIA(Root) - 식별목적 - 물품 Class - 물품 DB - 촬영 방식
    
    if 1 <= int(df[6][i][:3]) <= 200: # 식별목적(대분류)
        main_cat = "위해물품"
    elif 201 <= int(df[6][i][:3]) <= 300:
        main_cat = "정보저장매체"
    elif 301 <= int(df[6][i][:3]) <= 999:
        main_cat = "일반물품"
    else:
        main_cat = "error"

    object_class = df[5][i] # 물품 Class
    object_db = df[11][i] # 물품 DB

    if df[4][i] == "S": # 촬영 방식
        shot_type = "단일"
    elif df[4][i] == "P":
        shot_type = "복수"
    elif df[4][i] == "M":
        shot_type = "혼재"
    else:
        shot_type = "error"

    dir_main_cat = DIR_ROOT + "/" + main_cat
    dir_object_class = dir_main_cat + "/" + object_class
    dir_object_db = dir_object_class + "/" + object_db
    dir_shot_type = dir_object_db + "/" + shot_type

    if not os.path.exists(dir_main_cat):
        os.mkdir(dir_main_cat)
    if not os.path.exists(dir_object_class):
        os.mkdir(dir_object_class)
    if not os.path.exists(dir_object_db):
        os.mkdir(dir_object_db)
    if not os.path.exists(dir_shot_type):
        os.mkdir(dir_shot_type)

    # 이미지의 하단이 짤리는 현상 방지 코드
    img_detection = cv2.imread(markfile, cv2.IMREAD_COLOR) # image detection code
    gray = cv2.cvtColor(img_detection, cv2.COLOR_BGR2GRAY) # 이미지 그레이 변환
    blur = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=0)  # 가우시안 블러 사용 -> 이미지를 부드럽게 하기에 배경과 원하지 않는 부분을 제거하는 효과
    
    redlalwlt, th1 = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV) # 임계값 설정
    binary = cv2.bitwise_not(th1)

    contours, hierarchy = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # contours -> 3차원
    contours_xy = np.array(contours) # 윤곽선 최소 좌표를 구하기 위해 변형

    value = list()

    for x in range(len(contours_xy)): 
        for y in range(len(contours_xy[x])):
            value.append(contours_xy[x][y][0][1]) # height 값

    y_min = min(value) # height 시작 위치 조정
    
    if (y_min - 55 <= 0):
        y_min = 0
    else:
        y_min = y_min - 55
    
    img = img[y_min:y_min+HEIGHT, :]

    if os.path.exists(DIR_WORKING_COPY + '/' + file): # 이미지 변환이 끝난 raw 파일은 필요 없으므로 삭제
        os.remove(DIR_WORKING_COPY +'/'+ file)
    
    os.chdir(dir_shot_type) # 물품 분류 체계에 따른 최종 디렉토리로 이동
    cv2.imwrite(markfile, img) # 저장
    

    # 가변 변수 1 증가  
    i = i + 1

# 마지막 레코드 -> i 시작 번호에 복사해서 붙여넣기
print(i)


00012119_0.raw
E3S690-20220810-012119_S_Pistol_001-001_1.png
12119


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


00012119_1.raw
E3S690-20220810-012119_S_Pistol_001-001_2.png
12119
00012119_2.raw
E3S690-20220810-012119_S_Pistol_001-001_3.png
12119
00012119_3.raw
E3S690-20220810-012119_S_Pistol_001-001_4.png
12119
00012119_4.raw
E3S690-20220810-012119_S_Pistol_001-001_5.png
12119
00012119_5.raw
E3S690-20220810-012119_S_Pistol_001-001_6.png
12119
00012119_6.raw
E3S690-20220810-012119_S_Pistol_001-001_7.png
12119
00012119_7.raw
E3S690-20220810-012119_S_Pistol_001-001_8.png
12119
00012119_8.raw
E3S690-20220810-012119_S_Pistol_001-001_9.png
12119
00012120_0.raw
E3S690-20220810-012120_S_Pistol_002-001_1.png
12120
00012120_1.raw
E3S690-20220810-012120_S_Pistol_002-001_2.png
12120
00012120_2.raw
E3S690-20220810-012120_S_Pistol_002-001_3.png
12120
00012120_3.raw
E3S690-20220810-012120_S_Pistol_002-001_4.png
12120
00012120_4.raw
E3S690-20220810-012120_S_Pistol_002-001_5.png
12120
00012120_5.raw
E3S690-20220810-012120_S_Pistol_002-001_6.png
12120
00012120_6.raw
E3S690-20220810-012120_S_Pistol_002-001_7.png
1

In [ ]:
# 실사 이미지 네이밍 규칙 변경 v1.0
import shutil # 파일, 폴더 복사 모듈
import os
import numpy as np
import pandas as pd
import natsort # 숫자로 된 파일명 정렬 모듈

df = pd.read_excel('C:\\Users\\user\\Desktop\\[NIA] Runlist_1cycle_E3S690_Rev1_Draft.xlsx', header=None) # 엑셀 파일 위치 지정

DIR_DESKTOP = "C:\\Users\\user\\Desktop"
DIR_WORKING = DIR_DESKTOP + "\\Photo Data_Draft" # 촬영 데이터가 모여져 있는 폴더 경로
DIR_WORKING_RESULT = DIR_WORKING + '/' + DIR_WORKING.split('\\')[-1]

if not os.path.exists(DIR_WORKING_RESULT):
    shutil.copytree(DIR_WORKING, DIR_WORKING_RESULT) # 원본 폴더(dir_working)는 보관용으로 남기고, copy 폴더를 생성하여 작업.
    
i = 1 # 파일 명 찾는 가변 변수
j = 0

filelist = os.listdir(DIR_WORKING_RESULT)

filelist = natsort.natsorted(filelist)

for file in filelist:

    newname = df.values[i + j*9][10][:-2] + ".jpg"

    os.chdir(DIR_WORKING_RESULT) # change directory
    os.rename(DIR_WORKING_RESULT + "/" + file, DIR_WORKING_RESULT + "/" + newname)

    print(file)
    print(newname)
    print("-------------")

    j = j + 1

print(i + j * 9)